## OSU G2G Bandwidth Benchmark with MPI4Py
In this example we use [IPCMagic](https://github.com/eth-cscs/ipcluster_magic/tree/master) to run a test from the [OSU Bandwidth benchmark](http://mvapich.cse.ohio-state.edu/benchmarks/) with MPI4Py from a Jupyter notebook.
Using [this example](https://mpi4py.readthedocs.io/en/stable/tutorial.html#cuda-aware-mpi-python-gpu-arrays), we adapted the [osu_bw.py](https://github.com/mpi4py/mpi4py/blob/d0228f0397403ff73d8f41d90d97b411efda6128/demo/osu_bw.py) script from the MPI4Py repository so it uses an array allocated on the GPU.

* From a shell in Piz Daint this can be run using this Slurm job script:
 
```
#!/bin/bash -l

#SBATCH --job-name=osubw
#SBATCH --time=00:05:00
#SBATCH --nodes=2
#SBATCH --ntasks-per-core=1
#SBATCH --ntasks-per-node=1
#SBATCH --cpus-per-task=12
#SBATCH --partition=normal
#SBATCH --constraint=gpu
#SBATCH --account=<project>

# source python environment with cupy and mpi4py

export MPICH_RDMA_ENABLED_CUDA=1

srun python osu_bw_cupy.py
```

In [1]:
import os
import ipcmagic

In [2]:
os.environ['MPICH_RDMA_ENABLED_CUDA'] = '1'  # Enable direct communication between GPUs

In [3]:
%ipcluster --version

1.1.0


In [4]:
%ipcluster start -n 2

100%|██████████| 2/2 [00:06<00:00,  3.18s/engine]


In [5]:
# Disable IPyParallel's progress bar
%pxconfig --progress-after -1

In [6]:
%%px
import socket

socket.gethostname()

Out[1:1]: 'nid07225'

Out[0:1]: 'nid07225'

In [7]:
%%px
from osu_bw_cupy import osu_bw

In [ ]:
%%px
osu_bw()

[0:execute]
---------------------------------------------------------------------------
CUDARuntimeError                          Traceback (most recent call last)
Cell In[3], line 1
----> 1 osu_bw()

File /scratch/snx3000/class272/SummerUniversity2024/pyhpc/mpi/osu_bw_cupy.py:32, in osu_bw(BENCHMARH, skip, loop, window_size, skip_large, loop_large, window_size_large, large_message_size, MAX_MSG_SIZE)
     29         errmsg = None
     30     raise SystemExit(errmsg)
---> 32 s_buf = cp.arange(MAX_MSG_SIZE, dtype='i')
     33 r_buf = cp.empty_like(s_buf)
     34 cp.cuda.get_current_stream().synchronize()

File /apps/daint/UES/6.0.UP04/sandboxes/sarafael/hpcpy2023/lib/python3.9/site-packages/cupy/_creation/ranges.py:55, in arange(start, stop, step, dtype)
     52     else:
     53         return cupy.array([start], dtype=numpy.bool_)
---> 55 ret = cupy.empty((size,), dtype=dtype)
     56 typ = numpy.dtype(dtype).type
     57 _arange_ufunc(typ(start), typ(step), ret, dtype=dtype)

File /a

[stdout:1] # MPI G2G Bandwidth Test
# Size [B]    Bandwidth [MB/s]


Received Keyboard Interrupt. Sending signal SIGINT to engines...


In [ ]:
%ipcluster stop